In [ ]:
import sys
from src.chords_detection import ChordsAnalizer
from src.features_recognizer import FeaturesRecognizer
import librosa, librosa.display
import matplotlib as plt

HOP_LENGTH = 256
BUFF_SIZE = 1024 * 8
CHANNELS = 1 if sys.platform == "darwin" else 2
SAMPLE_RATE = 44100

user_name = 'test_user'
record_name = input('Please specify the record name.')

with open(f'./streamed_chords/{record_name}.csv', 'w+', encoding='utf-8') as csv_output:
    chords_recognizer = FeaturesRecognizer(hop_length=256)
    chords_analyzer = ChordsAnalizer(chords_recognizer, BUFF_SIZE, CHANNELS, SAMPLE_RATE)
    try:
        chords_analyzer.record_chords(csv_output, user_name, record_name)
    except KeyboardInterrupt:
        output_path = chords_analyzer.stop_audio_stream()
        print('Saved to:', csv_output.name)
        
final_chroma = chords_analyzer.build_final_chroma(output_path)

In [ ]:
def build_final_chroma(path):
    """
    Generate chroma feature from an audio file.
    Args:
        path (str): Path to the audio file.   
    Returns:
        np.ndarray: Chroma feature array.
    """
    y, sr = librosa.load(path)
    y_trimmed, _ = librosa.effects.trim(y, top_db=10)
    return librosa.feature.chroma_cqt(y=y_trimmed, sr=sr)

final_chroma = build_final_chroma(output_path)

def plot_chroma(chorma, name, cmap='coolwarm'):
    """
    Plot chroma feature.
    Args:
        chorma (np.ndarray): Chroma feature array.
        name (str): Name of the chroma feature.
        cmap (str): Color map.
    """
    plt.figure(figsize=(12, 3))
    librosa.display.specshow(chorma, x_axis='time', y_axis='chroma', hop_length=HOP_LENGTH, cmap=cmap)
    plt.colorbar()
    plt.title(f'Chroma {name} ({cmap})')
    plt.tight_layout()
    plt.show()
    
plot_chroma(final_chroma, 'CQT')